&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)

[Previous Notebook](02-Intro_to_cuDF_UDFs.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](01-Intro_to_cuDF.ipynb)
[2](02-Intro_to_cuDF_UDFs.ipynb)
[3]

# Applying CuDF: Exercise

Welcome to third cuDF tutorial notebook! This is a practical example that utilizes cuDF and cuPy, geared primarily for new users. The purpose of this tutorial is to introduce new users to a data science processing pipeline using RAPIDS on real life datasets. We will be working on a data science problem: US Accidents Prediction. This is a countrywide car accident dataset, which covers 49 states of the USA. The accident data are collected from February 2016 to June 2020, using two APIs that provide streaming traffic incident (or event) data. These APIs broadcast traffic data captured by a variety of entities, such as the US and state departments of transportation, law enforcement agencies, traffic cameras, and traffic sensors within the road-networks. Currently, there are about 3.5 million accident records in this dataset. 


## What should I do?

Given below is a complete data science preprocessing pipeline for the dataset using Pandas and Numpy libraries. Using the methods and techniques from the previous notebooks, you have to convert this pipeline to a a RAPIDS implementation, using CuDF and CuPy. Don't forget to time your code cells and compare the performance with this original code, to understand why we are using RAPIDS. If you get stuck in the middle, feel free to refer to this sample solution. 

## Here is the list of exercises in the lab where you need to modify code:
- <a href='#ex1'>Exercise 1</a><br> Loading the dataset from a csv file `'../../data/data.csv'` and store in a CuDF dataframe.
- <a href='#ex2'>Exercise 2</a><br> Creating kernel functions to run the given function optimally on a GPU.


The first step is downloading the dataset and putting it in the data directory, for using in this tutorial.
Download the dataset here, and place it in (host/data) folder. Now we will import the necessary libraries.

In [ ]:
import os
import cudf
import numpy as np
import cupy as cp
import math
np.random.seed(12)

<a id='ex1'></a>

First we need to load the dataset from the csv into CuDF dataframes, for the preprocessing steps. If you need help, refer to the Getting Data In and Out module from this [notebook](01-Intro_to_cuDF.ipynb/).

In [ ]:
#Modify the code in this cell

# Use cudf to read csv
%time df =                        
print(df)


First we will analyse the data and observe patterns that can help us process the data better for feeding to the machine learning algorithms in the future. By using the describe, we will generate the descriptive statistics for all the columns. Descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset’s distribution, excluding NaN values.

In [ ]:
df.describe()

We will check the size of the dataset that is to be processed using the len function.

In [ ]:
len(df)

You will notice that the dataset has 3513616 rows and takes quite a lot of time to read from the file. As we go ahead with the preprocessing, computations will require more time to execute, and that's where the RAPIDS comes to the rescue!

Now we use the info function to check the datatype of all the columns in the dataset.

In [ ]:
df.info()

We will also check the number of missing values in the dataset, so that we can drop or fill in the missing values.

In [ ]:
df.isna().sum()

There are many columns with null values, and we will fill them with random values or the mean from the column. We will drop some text columns, as we are not doing any natural language processing right now, but feel free to explore them on your own. We will also drop the columns with too many Nans as filling them will throw our accuracy.

In [ ]:
df = df.drop(columns = ['ID','Start_Time','End_Time','Street','Side','Description','Number','City','Country','Zipcode','Timezone','Airport_Code','Weather_Timestamp','Wind_Chill(F)','Wind_Direction','Wind_Speed(mph)','Precipitation(in)'])

In [ ]:
#Here we are filling the TMC with mean.
df['TMC'] = df['TMC'].fillna(df['TMC'].mean())
df['End_Lat'] = df['End_Lat'].fillna(df['End_Lat'].mean())
df['End_Lng'] = df['End_Lng'].fillna(df['End_Lng'].mean())
df['Temperature(F)'] = df['Temperature(F)'].fillna(df['Temperature(F)'].mean())
df['Humidity(%)'] = df['Humidity(%)'].fillna(df['Humidity(%)'].mean())
df['Pressure(in)'] = df['Pressure(in)'].fillna(df['Pressure(in)'].mean())
df['Visibility(mi)'] = df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean())
df['Humidity(%)'] = df['Humidity(%)'].fillna(df['Humidity(%)'].mean())
df['Pressure(in)'] = df['Pressure(in)'].fillna(df['Pressure(in)'].mean())
df['Visibility(mi)'] = df['Visibility(mi)'].fillna(df['Visibility(mi)'].mean())


df['Weather_Condition'] = df['Weather_Condition'].fillna('Fair')
df['Sunrise_Sunset'] = df['Sunrise_Sunset'].fillna('Day')
df['Civil_Twilight'] = df['Civil_Twilight'].fillna('Day')
df['Nautical_Twilight'] = df['Nautical_Twilight'].fillna('Day')
df['Astronomical_Twilight'] = df['Astronomical_Twilight'].fillna('Day')
df['Weather_Condition'] = df['Weather_Condition'].fillna('Fair')

Now all the columns contain no Nan values and we can go ahead with the preprocessing.

<a id='ex2'></a>
       
As you have observed in the dataset we have the start and end coordinates,so  let us apply Haversine distance formula to get the accident coverage distance. Take note of how these functions use the row-wise operations, something that we have learnt before. If you need help while creating the user defined functions refer to this [notebook](02-Intro_to_cuDF_UDFs.ipynb).

In [ ]:
from math import cos, sin, asin, sqrt, pi, atan2
from numba import cuda

In [ ]:
#Modify the code in this cell

def haversine_distance_kernel(Start_Lat, Start_Lng, End_Lat, End_Lng, out):
 
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(Start_Lat, Start_Lng, End_Lat, End_Lng)):
 
     #Perform the computations here and store the final value in out[i]
              
        out[i] = 

In [ ]:
#Modify the code in this cell

%%time
#Add the arguments to the apply_rows function for the haversine distance kernel
df = df.apply_rows()

Wow! The code segment that previously took  7 minutes to compute, now gets executed in less than a second! 

In [ ]:
#Modify the code in this cell

def haversine_distance_kernel(Start_Lat, Start_Lng, End_Lat, End_Lng, out):
 
    for i, (x_1, y_1, x_2, y_2) in enumerate(zip(Start_Lat, Start_Lng, End_Lat, End_Lng)):
 
 #Perform the computations here and store the final value in out[i]
        
        out[i] = 

In [ ]:
#Modify the code in this cell

%%time
#Add the arguments to the apply_chunks function for the haversine distance kernel
outdf = df.apply_chunks()

Save the dataframe in a csv for future use, and make sure you refer to our sample solution and compared your code's performance with it.

In [ ]:
df.head()

In [ ]:
df = df.dropna()

In [ ]:
df.to_csv("../../data/data_proc.csv")

# Conclusion 

Thus we have successfully used CuDF and CuPy to process the accidents dataset, and converted the data to a form more suitable to apply machine learning algorithms. In the extra labs for future labs in CuML we will be using this processed dataset. You must have observed the parallels between the RAPIDS pipeline and traditional pipeline while writing your code. Try to experiment with the processing and making your code as efficient as possible. If you faced any difficulties in solving this exercise, refer to our sample solution notebook [here]((04-Cudf_Solution.ipynb)).

# References

- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, and Rajiv Ramnath. “A Countrywide Traffic Accident Dataset.”, 2019.

- Moosavi, Sobhan, Mohammad Hossein Samavatian, Srinivasan Parthasarathy, Radu Teodorescu, and Rajiv Ramnath. "Accident Risk Prediction based on Heterogeneous Sparse Data: New Dataset and Insights." In proceedings of the 27th ACM SIGSPATIAL International Conference on Advances in Geographic Information Systems, ACM, 2019.

- If you need to refer to the dataset, you can download it [here](https://www.kaggle.com/sobhanmoosavi/us-accidents).

<center><a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a></center><br />

- This dataset is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

## Licensing
  
This material is released by OpenACC-Standard.org, in collaboration with NVIDIA Corporation, under the Creative Commons Attribution 4.0 International (CC BY 4.0).

[Previous Notebook](02-Intro_to_cuDF_UDFs.ipynb)
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
[1](01-Intro_to_cuDF.ipynb)
[2](02-Intro_to_cuDF_UDFs.ipynb)
[3]

&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&emsp;&emsp;&emsp;
&emsp;&emsp;&ensp;
[Home Page](../START_HERE.ipynb)